In [ ]:


from agent.OnOff_Agent import OnOffAgent
from environment.ContinuousEnvironment import ContinuousSimpleEnvironment

%load_ext autoreload
%autoreload 2

In [4]:
from Performance import load_trained_agent
from Performance import open_json_params
from agent.DQN_Agent import DQNAgent
from environment.DiscreteEnvironment import DiscreteSimpleEnvironment

results_path = r"C:\Users\hbenoit\Desktop\DIET_Controller\logs\simple_simulation\DQN_Agent\results\2022_6_15\results_2022_6_15_7_31"
d= open_json_params(results_path)

print(d)

agent = DQNAgent(DiscreteSimpleEnvironment(discrete_action_dim=d["discrete_action_dim"]))

agent = load_trained_agent(agent,results_path=results_path)

{'is_test': False, 'memory_size': 2000, 'batch_size': 64, 'target_update': 100, 'actor_update': 4, 'epsilon_decay': 5e-05, 'max_epsilon': 1, 'min_epsilon': 0.0, 'lr': 0.001, 'gamma': 0.99, 'inside_dim': 128, 'num_hidden_layers': 14, 'seed': 800, 'param_list': ['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'], 'observation_dim': 6, 'action_dim': 1, 'min_temp': 16, 'max_temp': 21, 'alpha': 1, 'beta': 1, 'modelname': 'CELLS_v1.fmu', 'days': 151, 'hours': 24, 'minutes': 60, 'seconds': 60, 'ep_timestep': 6, 'discrete_action_dim': 100, 'num_episodes': 10, 'num_iterations': 21744, 'final_reward': 242275.87176397615, 'final_cumulative_heating': 11472257911.653708, 'pmvs': {'[-inf,-2]': 0.10946008094186901, '[-2.0,-1.5]': 0.1803991905813098, '[-1.5,-1.0]': 0.18393119941133187, '[-1.0,-0.5]': 0.28969830757910225, '[-0.5,0.0]': 0.13544426048565122, '[0.0,0.5]': 0.06733351729212657, '[0.5,1.0]': 0.03103384841795438, '[1.0,inf]': 0.0026995952906548933}}


RuntimeError: Error(s) in loading state_dict for Network:
	Missing key(s) in state_dict: "layers.6.weight", "layers.6.bias", "layers.8.weight", "layers.8.bias", "layers.10.weight", "layers.10.bias", "layers.12.weight", "layers.12.bias", "layers.14.weight", "layers.14.bias", "layers.16.weight", "layers.16.bias", "layers.18.weight", "layers.18.bias", "layers.20.weight", "layers.20.bias", "layers.22.weight", "layers.22.bias", "layers.24.weight", "layers.24.bias", "layers.26.weight", "layers.26.bias", "layers.28.weight", "layers.28.bias", "layers.30.weight", "layers.30.bias". 
	size mismatch for layers.4.weight: copying a param with shape torch.Size([100, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for layers.4.bias: copying a param with shape torch.Size([100]) from checkpoint, the shape in current model is torch.Size([128]).

In [ ]:
env = ContinuousSimpleEnvironment(param_list=['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
alpha=0.5,
beta=1,
min_temp=16,
max_temp=21,
modelname='CELLS_v1.fmu',
simulation_path=r'C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\EnergyPlus_simulations\simple_simulation',
days=151,
hours=24,
minutes=60,
seconds=60,
ep_timestep=6)


logging_path = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\logs\simple_simulation"


agent = OnOffAgent(env=env)


agent.train(logging_path=logging_path, num_episodes=1, num_iterations=200, log=True)